In [8]:
from ollama import chat
from pydantic import BaseModel, Field
import json
import numpy as np
from typing import List


#word 'pairs'
sentiment_word_pairs = ["Uplifting", "Depressing", 
    "Triumphant", "Defeated", 
    "Thrilling", "Boring",
    "Inspiring", "Discouraging",
    "Comforting", "Disturbing",
    "Amusing", "Somber",
    "Profound", "Shallow",
    "Intimate", "Distant",
    "Redemptive", "Condemning",
    "Serene", "Chaotic",
    "Harmonious", "Discordant",
    "Nostalgic", "New",
    "Memorable", "Forgettable",
    "Popular", "Niche",
    "Spirited", "Lifeless",
    "Provocative", "Innocuous",
    "Jubilant", "Despondent",
    "Surreal", "Realistic",
    "Immerseive", "Detached",
    "Fantastical", "Grounded",
    "Complex", "Simple",
    "Innovative", "Common",
    "Raw", "Polished",
    "Ambitious", "Unimaginative",
    "Liberating", "Confining",
    "Childish", "Mature"
    ]

#movie values 
sentiment_word_pairs_values = [0.8721, 0.8234, 0.7482, 0.8412, 0.9123, 0.7654, 0.6321, 0.8123, 
    0.8567, 0.7245, 0.7934, 0.9021, 0.8945, 0.9278, 0.8932, 0.7345, 
    0.8456, 0.7987, 0.8712, 0.8129, 0.6523, 0.7268, 0.5124, 0.7845, 
    0.8492, 0.3127, 0.8721, 0.8234, 0.7482, 0.8412, 0.9123, 0.7654, 
    0.6321, 0.8123, 0.8567, 0.7245, 0.7934, 0.9021, 0.8945, 0.9278, 
    0.8932, 0.7345, 0.8456, 0.7987, 0.8712, 0.8129, 0.6523, 0.7268, 
    0.5124, 0.7845]

#niche/related words
related_words = ["Whimsical",
    "Nostalgic",
    "Heartfelt",
    "Invigorating",
    "Earnest",
    "Homespun",
    "Liberating",
    "Idyllic",
    "Melancholic",
    "Measured",
    "Folksy",
    "Isolated",
    "Stubborn"]

#model 4 generates songs
from pydantic import BaseModel, Field
from typing import List

class SongsList:
    @classmethod
    def model_json_schema(cls):
        return {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "song_title": {"type": "string", "description": "The title of the song"},
                    "artist": {"type": "string", "description": "The name of the artist/band who performed the song"}
                },
                "required": ["song_title", "artist"]
            },
            "description": "A list of songs with their titles and artists"
        }


prompt = f"""
    You are given a list of related words: {related_words}.  
    Using these words, return songs whose lyrics match their themes.  

    Only return a JSON object. No extra text.  
    Format the output strictly as follows:  
    {{"song_title": "Song Name", "artist": "Artist Name"}},

    Each song must include the correct artist.
    Choose songs based on lyrics, not just the title.
    """

response = chat(
    messages=[
        {
            "role": "system",
            "content": prompt
        },
        {
            "role": "user",
            "content": f"{related_words} are the related words. Please generate songs and their respective song artists."
        }
    ],
    model='llama3.2:latest',
    format=SongsList.model_json_schema(),
)
    
songs_data = json.loads(response.message.content)

print(songs_data)


#GENERATE SONGS BY FEELING/SENTIMENT
prompt = f"""
    You are given a list of related words: {related_words}.  
    Using these words, return songs whose sentiment, mood, or general vibe match their themes.  

    Only return a JSON object. No extra text.  
    Format the output strictly as follows:  
    {{"song_title": "Song Name", "artist": "Artist Name"}},

    Each song must include the correct artist.
    Do not choose just based on the song title.
    """

response = chat(
    messages=[
        {
            "role": "system",
            "content": prompt
        },
        {
            "role": "user",
            "content": f"{related_words} are the related words. Please generate songs and their respective song artists."
        }
    ],
    model='llama3.2:latest',
    format=SongsList.model_json_schema(),
)
    
songs_data = json.loads(response.message.content)
print(songs_data)

    #GENERATE SONGS W/OUT EXTRA INSTRUCTION
prompt = f"""
    You are given a list of related words: {related_words}.  
    Using these words, return songs that are similar.  

    Only return a JSON object. No extra text.  
    Format the output strictly as follows:  
    {{"song_title": "Song Name", "artist": "Artist Name"}},

    Each song must include the correct artist.
    """

response = chat(
    messages=[
        {
            "role": "system",
            "content": prompt
        },
        {
            "role": "user",
            "content": f"{related_words} are the related words. Please generate songs and their respective song artists."
        }
    ],
    model='llama3.2:latest',
    format=SongsList.model_json_schema(),
)
    
songs_data = json.loads(response.message.content)
print(songs_data)


#model 5 does cosine sim 
#first, generate the 50 number vectors 
#attempt here to do it with one song

first_song = songs_data[0]
print(f"The first song is: {first_song}")


class SentimentScores:
    @classmethod
    def model_json_schema(cls):
        return {
            "type": "array",
            "items": {
                "type": "number",
                "minimum": 0,
                "maximum": 1
            },
            "minItems": 50,
            "maxItems": 50,
            "description": "A list of 50 sentiment scores between 0 and 1"
        }

sentiment_response = chat(
    messages=[
        {"role": "system", "content": prompt},
        {"role": "user", "content": f"""{sentiment_word_pairs} are 50 sentiment words. 
        Please generate a list of 50 numerical sentiment scores (a number from 0 to 1) 
        based on each of those words for the song {first_song}.
        Return it as a list of numbers."""}
    ],
    model='llama3.2:latest',
    format=SentimentScores.model_json_schema(),  
)
sentiment_data = json.loads(sentiment_response.message.content)
print(sentiment_data)

#cosine sim 
a = np.array(sentiment_word_pairs_values) #movie 
b = np.array(sentiment_data) #song

#print(a.shape)
#print(b.shape)
print("Cos sim is: ")
print((a @ b.T) / (np.linalg.norm(a)*np.linalg.norm(b)))

[{'song_title': 'Pure Imagination', 'artist': 'Les Misérables'}, {'song_title': 'American Pie', 'artist': 'Don McLean'}, {'song_title': 'Hallelujah', 'artist': 'Jeff Buckley'}, {'song_title': 'I Will Follow You into the Dark', 'artist': 'Death Cab for Cutie'}, {'song_title': 'Home', 'artist': 'Edward Sharpe & The Magnetic Zeros'}, {'song_title': 'Landslide', 'artist': 'Fleetwood Mac'}, {'song_title': 'Stay with Me', 'artist': 'Sam Smith'}, {'song_title': "I'm Yours", 'artist': 'Jason Mraz'}, {'song_title': 'Ho Hey', 'artist': 'The Lumineers'}, {'song_title': 'Best Day of My Life', 'artist': 'American Authors'}, {'song_title': 'Skinny Love', 'artist': 'Bon Iver'}, {'song_title': 'First Day of My Life', 'artist': 'Bright Eyes'}, {'song_title': 'Wagon Wheel', 'artist': 'Darius Rucker'}]
[{'song_title': 'Pure Morning', 'artist': 'Edwyn Collins'}, {'song_title': 'Everlong', 'artist': 'Foo Fighters'}, {'song_title': 'Hallelujah', 'artist': 'Jeff Buckley'}, {'song_title': 'Mr. Blue Sky', 'art